In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from datasets import TripletMNIST

from trainer import fit
import numpy as np
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from networks import EmbeddingNet, TripletNet
from torchvision.datasets import ImageFolder
from losses import TripletLoss
from metrics import AccumulatedAccuracyMetric

In [3]:
dataset = ImageFolder("../data/teacher_data/Dishes")

In [4]:
len(dataset)

1200

In [5]:
from torch.utils.data import random_split

test_size = 0.2
test_n_samples = int(len(dataset)*test_size)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len(dataset)-test_n_samples, test_n_samples])

In [6]:
len(val_dataset)

240

In [7]:
cuda = torch.cuda.is_available()

In [8]:
import numpy as np

In [9]:
train_dataset.dataset

Dataset ImageFolder
    Number of datapoints: 1200
    Root location: ../data/teacher_data/Dishes

In [10]:
from torch.utils.data import random_split



test_size = 0.2
test_n_samples = int(len(dataset)*test_size)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len(dataset)-test_n_samples, test_n_samples])

triplet_dataset_train = TripletMNIST(train_dataset.dataset, train=True, subset_indices=train_dataset.indices) 
triplet_dataset_val = TripletMNIST(val_dataset.dataset, train=False, subset_indices=val_dataset.indices)
# triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 128
kwargs = {'num_workers': 10, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_dataset_train, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_dataset_val, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)

if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 1
log_interval = 100

In [17]:
fit(triplet_train_loader, triplet_test_loader,  model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AccumulatedAccuracyMetric()])

0
[]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/yurykasimov/Library/Caches/pypoetry/virtualenvs/google-kaggle-ETRbyZYM-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/2_/8wzmxgd96lz31gp705ghksnr0000gn/T/ipykernel_6971/2703274610.py", line 1, in <cell line: 1>
    fit(triplet_train_loader, triplet_test_loader,  model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AccumulatedAccuracyMetric()])
  File "/Users/yurykasimov/Documents/google-kaggle/notebooks/trainer.py", line 23, in fit
    train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)
  File "/Users/yurykasimov/Documents/google-kaggle/notebooks/trainer.py", line 61, in train_epoch
    outputs = model(*data)
  File "/Users/yurykasimov/Library/Caches/pypoetry/virtualenvs/google-kaggle-ETRbyZYM-py3.10/lib/python3.10/site-packages/torch/nn/modu

In [ ]:
for batch_idx, (data, target) in enumerate(triplet_test_loader):
    print((data[0].shape))
    break

In [26]:
for batch_idx, (data, target) in enumerate(triplet_train_loader):
    print((data[2]).shape)
    break

torch.Size([128, 3, 224, 224])


In [24]:
data[0].view(data[0].size(0),-1).shape

torch.Size([3, 50176])

In [14]:
embedding_test = EmbeddingNet()
embedding_test(data[0])

torch.Size([64, 53, 53])
torch.Size([64, 2809])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x2809 and 179776x256)

In [14]:
3392*53

179776